# Imports

In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
!pip install sweetviz
import sweetviz as sv
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Reading the Files

In [ ]:
train_1 = pd.read_csv('../input/av-janatahack-machine-learning-in-agriculture/train_yaOffsB.csv')
test_1 = pd.read_csv('../input/av-janatahack-machine-learning-in-agriculture/test_pFkWwen.csv')

In [ ]:
train_1

In [ ]:
s = sns.pairplot(train_1)

In [ ]:
train_1['type'] = 0
test_1['Crop_Damage'] = 0
test_1['type'] = 1
master_1 = pd.concat([train_1,test_1])

In [ ]:
master = master_1.copy()

In [ ]:
master['Number_Weeks_Used'] = master['Number_Weeks_Used'].fillna(0)
master['Total_Doses'] = master['Number_Doses_Week']*master['Number_Weeks_Used']
#master['Total_Skipped'] = master['Number_Doses_Week']*master['Number_Weeks_Quit']
#master['Used_Skipped_Ratio'] = master['Number_Weeks_Quit']/master['Number_Weeks_Used']
#master['doses_by_insect'] = master['Number_Doses_Week']/master['Estimated_Insects_Count']
#master['used_by_insect'] = master['Number_Weeks_Used']/master['Estimated_Insects_Count']
master['quit_by_insect'] = master['Number_Weeks_Quit']/master['Estimated_Insects_Count']
master['crop_by_insect'] = master['Crop_Type']/master['Estimated_Insects_Count']
master['soil_by_insect'] = master['Soil_Type']/master['Estimated_Insects_Count']
#master['pesticide_by_insect'] = master['Pesticide_Use_Category']/master['Estimated_Insects_Count']
#master['season_by_insect'] = master['Season']/master['Estimated_Insects_Count']
master['Total_Weeks'] = master['Number_Weeks_Quit']+master['Number_Weeks_Used']

In [ ]:
def create_aggregates_count(df,colagg,coltar,nsuffix):

    dfAg=df.dropna (subset=coltar).groupby (by=colagg).agg({coltar[0]:['count']})
    lstColsAg=dfAg.reset_index().columns.to_list() 
    print(lstColsAg)                                                           
    colsAg=[] 
    for n1 in range (len(lstColsAg)):
        if len(lstColsAg[n1][1])>0:
            colname=lstColsAg[n1][0]+"_"+lstColsAg[n1][1]+"_"+str(nsuffix)+str(len(colagg))
            colsAg.append(colname) 
    dfAg.columns=colsAg 
    dfAg=dfAg.reset_index() 
    df=df.merge(dfAg, how='left', on=colagg) 
    return df

def create_aggregates(df,colagg,coltar,nsuffix):
    ##coltar and colagg should both be passed as lists, but len(coltar) = 1 
    dfAg=df.dropna(subset=coltar).groupby(by=colagg).agg({coltar[0]:['median','mad','min','max', 'std', 'var','sem','skew', 'quantile']})#,'median','mad','min','max', 'std', 'var','sem','skew', 'quantile']})#,'median','mad','min','max', 'std', 'var','sem','skew', 'quantile'],})#,'median','mad','min','max', 'std', 'var','sem','skew', 'quantile']})

    lstColsAg=dfAg.reset_index().columns.to_list() 
    print(lstColsAg)                                                           
    colsAg=[] 
    for n1 in range (len(lstColsAg)):
        if len(lstColsAg[n1][1])>0:
            colname=lstColsAg[n1][0]+"_"+lstColsAg[n1][1]+"_"+str(nsuffix)+str(len(colagg))
            colsAg.append(colname) 
    dfAg.columns=colsAg 
    dfAg=dfAg.reset_index() 
    df=df.merge(dfAg, how='left', on=colagg) 
    return df

In [ ]:
colstar = [['Estimated_Insects_Count'],['Number_Weeks_Quit'],['Number_Weeks_Used'],['Number_Doses_Week']]
colsagg = ['Crop_Type','Soil_Type','Pesticide_Use_Category','Season',['Crop_Type','Soil_Type'],
           ['Soil_Type','Pesticide_Use_Category'],['Pesticide_Use_Category','Season'],['Crop_Type','Pesticide_Use_Category'],
           ['Crop_Type','Season'],['Soil_Type','Season'],['Crop_Type','Soil_Type','Pesticide_Use_Category','Season']]
n = 0
for coltar in colstar:
    for colagg in colsagg:
        master = create_aggregates_count(master,colagg,coltar,n)
        n = n+1

In [ ]:
from sklearn.preprocessing import power_transform
master['Estimated_Insects_Count_T'] = power_transform(master['Estimated_Insects_Count'].values.reshape(-1, 1), method='box-cox')

In [ ]:
master.corr().to_csv('master_corr.csv')

In [ ]:
my_report = sv.analyze(master)
my_report.show_html() # Default arguments will generate to "SWEETVIZ_REPORT.html"

In [ ]:
master.columns

In [ ]:
cols_ignore = ['ID', 'Crop_Damage','type','Estimated_Insects_Count_T',
       'Estimated_Insects_Count_count_222', 'Estimated_Insects_Count_count_36',
       'Estimated_Insects_Count_count_52',
       'Estimated_Insects_Count_count_62', 'Estimated_Insects_Count_count_72',
       'Estimated_Insects_Count_count_92',
       'Estimated_Insects_Count_count_104',
       'Number_Weeks_Quit_count_1322',
       'Number_Weeks_Quit_count_146', 
       'Number_Weeks_Quit_count_162', 'Number_Weeks_Quit_count_172',
       'Number_Weeks_Quit_count_182', 
       'Number_Weeks_Quit_count_202', 'Number_Weeks_Quit_count_214',
       'Number_Weeks_Used_count_2422', 'Number_Weeks_Used_count_256',
       'Number_Weeks_Used_count_272',
       'Number_Weeks_Used_count_282', 'Number_Weeks_Used_count_292',
       'Number_Weeks_Used_count_312',
       'Number_Weeks_Used_count_324', 
       'Number_Doses_Week_count_3522',
       'Number_Doses_Week_count_366', 
       'Number_Doses_Week_count_382', 'Number_Doses_Week_count_392',
       'Number_Doses_Week_count_402', 
       'Number_Doses_Week_count_422', 'Number_Doses_Week_count_434',]
cols_y = ['Crop_Damage']
cols_X = list(set(master.columns) - set(cols_ignore))
X = master[cols_X][master['type']==0]
y = master[cols_y][master['type']==0]

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 42)
X_res, y_res = sm.fit_resample(X, y)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split( X, y, test_size=0.33, random_state = 0)

In [ ]:
def accuracy(y_true, y_pred):  
    preds = y_pred.reshape(len(np.unique(y_true)), -1)
    preds = preds.argmax(axis = 0)
    print(preds.shape)
    print(y_true.shape)
    return 'accuracy', accurcacy_score(y_true, preds,average='weighted'), True

In [ ]:
from sklearn.metrics import mean_squared_error
def rmse_mc(preds, train_data):  
    labels = train_data.get_label()
    #assert len(preds) == len(labels)
    thresh_1 = 0.5
    thresh_2 = 1.05
    preds = np.where(preds>thresh_2,2,np.where(preds>thresh_1,1,0))
    score = mean_squared_error(labels, preds)
    
    #terms_to_sum = [(math.log(labels[i] + 1) - math.log(preds[i] + 1)) ** 2.0 for i,pred in enumerate(labels)]
    return score#(sum(terms_to_sum) * (1.0/len(preds))) ** 0.5

In [ ]:
cw = {0:4.941,1:1.692,2:0.03}
clf = lgb.LGBMClassifier(num_leaves = 63,learning_rate = 0.005,
                         n_estimators = 1000,objective = 'multiclass',num_class = 3,
                         reg_lambda = 3, silent = False).fit(X_train,np.array(y_train))

In [ ]:
y_preds = clf.predict(X_val)

In [ ]:
print(accuracy_score(y_val,y_preds))

In [ ]:
from catboost import CatBoostClassifier
clf2 = CatBoostClassifier().fit(X_train,y_train)b

In [ ]:
y_preds = clf2.predict(X_val)
print(accuracy_score(y_val,y_preds))



params = {
          "objective" : "multiclass",
          "num_class" : 3,
          #"fobj": "accuracy",
          #"feval": "rmse_mc",
          #"boosting_type":"dart",
          "metric" : "multi_error",
           #"feval":"rmsle",
          "num_leaves" : 63,
            #"max_bin":7,
          "learning_rate" : 0.005,
          "bagging_fraction" : 0.7,
          "feature_fraction" : 0.5,
          "bagging_frequency" : 6,
          "bagging_seed" : 42,
          "lambda_l1":2,
          "lambda_l2":3,
          "verbose" : 1,
          "seed": 42,
          "max_bin":255 }

lgtrain, lgval = lgb.Dataset(X_train, y_train), lgb.Dataset(X_val, y_val)
lgbmodel = lgb.train(params, lgtrain, 2000, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, verbose_eval=300)

In [ ]:
def preds(cols,model,thresh_1 = 0.245,thresh_2 = 0.13):
    X_test = master[master['type'] == 1]
    X_test = X_test.reset_index().drop(['index'],axis = 1)
    y_id = pd.Series(X_test['ID'])
    X_test = X_test[cols]
    #print(X_test)
    #lgtest = lgb.Dataset(X_test)
    y_test = model.predict(X_test)#,num_iteration=model.best_iteration)
    y_probs = pd.DataFrame(data = y_test)
    #print('class 2',len(y_probs[2][y_probs[2]>thresh_2]))
    #print('class 1',len(y_probs[1][(y_probs[2]<=thresh_2)&(y_probs[1]>thresh_1)]))
    print(y_probs)
    #y_probs['Crop_Damage'] = np.where(y_probs[2]>thresh_2,2,np.where(y_probs[1]>thresh_1,1,0))
    #y_probs['Crop_Damage'] = y_probs.idxmax(axis = 1)
    y_df = pd.concat([y_id,y_probs[0]],axis = 1)
    print(y_df)
    y_df = y_df.rename({0:'Crop_Damage'},axis = 1)
    sub = y_df.to_csv('sub.csv', index = False)

    return sub
    